outlook|temp|humidity|windy|Play
-------|----|--------|-----|----
sunny  |hot |high    |False|no
rainy  |mild |low   |True |yes
overcast|hot|high    |False|yes
rainy  |cool|low     |True |no
overcast|hot|normal|False|yes


### Random Forest :

* Random forest is an ensemble classifier used to classify the output.
* Group of decision trees can be called as Random forest.
* We will consider the output of each decision tree and define the highest repeated times output as the predicted output.
* We mainly use 2 terms in random forest .
    * entropy  --- the measure of the impurity in the model.
        * It should be in the range of 0 to 1.
        *  <img src = "https://miro.medium.com/max/391/1*nNY_7_aWRwp8E2DyGduEPg.png">
    * Information Gain -- the measure of reducing entropy due to split 
        * It should be in the range of 0 to 1
        * <img src = "https://miro.medium.com/max/700/0*08CaHVjPCgs_fZyp" width = "200" height = "200">
    

In [5]:
#-[p(yes)*log(p(yes)) + p(no)*log(p(no))]
import numpy as np

-((3/6) * np.log(3/6) + (2/6) * np.log(2/6))

0.7127776865026758

In [7]:
### importing the libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
data = pd.read_csv("https://raw.githubusercontent.com/AP-Skill-Development-Corporation/Tirumala-ML/main/Day-9/diabetes.csv")

In [9]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [10]:
data.shape

(768, 9)

In [25]:
data.isna().count()

Pregnancies                 768
Glucose                     768
BloodPressure               768
SkinThickness               768
Insulin                     768
BMI                         768
DiabetesPedigreeFunction    768
Age                         768
Outcome                     768
dtype: int64

In [26]:
X = data.drop("Outcome",axis = 1)
y = data["Outcome"]

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 66)



In [29]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

C:\Users\Niharikaa\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
rfc_predict = rfc.predict(X_test)

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix

rfc_cv_score = cross_val_score(rfc,X,y,cv = 10,scoring = 'roc_auc')

print("===Confusion matric===")
print(confusion_matrix(y_test,rfc_predict))
print("===Classification report===")
print(classification_report(y_test,rfc_predict))
print('\n')
print("======All Auccary scores=====")
print(rfc_cv_score)
print("\n")
print("===mean accuracy score =====")
print(rfc_cv_score.mean())

===Confusion matric===
[[85 23]
 [25 21]]
===Classification report===
              precision    recall  f1-score   support

           0       0.77      0.79      0.78       108
           1       0.48      0.46      0.47        46

   micro avg       0.69      0.69      0.69       154
   macro avg       0.62      0.62      0.62       154
weighted avg       0.68      0.69      0.69       154



======All Auccary scores=====
[0.74555556 0.80962963 0.80148148 0.78037037 0.7362963  0.8262963
 0.82074074 0.84148148 0.77807692 0.82153846]


===mean accuracy score =====
0.7961467236467236


In [33]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
### number of trees in random forest

n_estimators = [int(x) for x in np.linspace(start = 200,stop = 2000,num = 10)]

## number of features at every split

max_features = ['auto','sqrt']
### Max_depth
max_depth = [int(x) for x in np.linspace(100,500,num = 11)]
## create random grid:

random_grid = {"n_estimators":n_estimators,"max_features":max_features,"max_depth":max_depth}

### random search parameters:

rfc_random = RandomizedSearchCV(estimator = rfc,param_distributions = random_grid,n_iter = 100,cv = 3,verbose = 2,random_state = 42,n_jobs = -1)

rfc_random.fit(X_train,y_train)
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.8min finished


{'n_estimators': 1800, 'max_features': 'sqrt', 'max_depth': 340}


In [34]:
rfc = RandomForestClassifier(n_estimators = 1800,max_features = 'sqrt',max_depth = 340)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc,X,y,cv = 10,scoring = 'roc_auc')

print("===Confusion matric===")
print(confusion_matrix(y_test,rfc_predict))
print("===Classification report===")
print(classification_report(y_test,rfc_predict))
print('\n')
print("======All Auccary scores=====")
print(rfc_cv_score)
print("\n")
print("===mean accuracy score =====")
print(rfc_cv_score.mean())

===Confusion matric===
[[93 15]
 [20 26]]
===Classification report===
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       108
           1       0.63      0.57      0.60        46

   micro avg       0.77      0.77      0.77       154
   macro avg       0.73      0.71      0.72       154
weighted avg       0.77      0.77      0.77       154



======All Auccary scores=====
[0.78481481 0.83111111 0.83074074 0.73703704 0.81703704 0.85851852
 0.86814815 0.90888889 0.80692308 0.85846154]


===mean accuracy score =====
0.8301680911680911
